# This is to scrape the HDB application rates data from various sources

In [111]:
# let's import some libraries

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [114]:
# Correct path to the ChromeDriver executable
service = Service('./chromedriver-win64/chromedriver.exe')  # Update the path if needed
driver = webdriver.Chrome(service=service)

In [115]:
# Open the Python website
driver.get("https://www.propertyguru.com.sg/property-guides/bto-application-rate-which-hdb-estate-easiest-62148")

In [206]:
def extract_data(text):
    """
    This function extracts the number of rooms from the given text.
    Args:
        text (str): The input text containing information about rooms.
    Returns:
        int: The number of rooms extracted from the text. And the number of applications.
    If no valid room number is found, it returns 0.
    """

    text = text.replace(')',' ').replace('s',' ').replace('e',' ').replace(' room','-room')
    # Split the text into words
    words = text.split()
    num_rm = ''

    for word in words:
        # Check if the word contains '-room' and extract the number
        if '-ro' in word:
            try:
                num_rm = (word.split('-')[0].strip())
            except ValueError:
                # If the number cannot be converted, skip this word
                continue
        
        if num_rm.isdigit() and word.replace(',', '').isdigit():
            # Return the number of rooms as an integer
            return int(num_rm), int(word.replace(',', ''))

    # Return 0 if no valid room number is found
    return 0,0

def extract_relevant(text_body_ls):

    project_dict = {}
    project_details = retrieve_project_details(text_body_ls)

    for i in text_body_ls:
        if i == '':
            continue
        num_rm, number = extract_data(i)
        if num_rm == 0 or number == 0:
            continue
        if num_rm not in project_dict:
            project_dict[num_rm] = [number]
        elif len(project_dict[num_rm]) > 2:
            return project_details,project_dict
            
        else:
            # Append the number to the list for the corresponding number of rooms
            project_dict[num_rm].append(number)
    
    return project_details,project_dict
        
    
def separate_by_mature(text_body_ls):
    """
    Separates items in the list into groups based on occurrences of 'Mature'.
    Args:
        text_body_ls (list): List of strings to process.
    Returns:
        list: A list of groups, where each group is a list of items between 'Mature' occurrences.
    """
    groups = []
    current_group = []

    for item in text_body_ls:
        # Check if the item contains 'Mature'
        if 'Mature' in item or 'Non-mature' in item or 'mature' in item or 'non-mature' in item:
            # If there's an existing group, add it to the groups list
            if current_group:
                groups.append(current_group)
                current_group = []
        # Add the current item to the current group
        current_group.append(item)

    # Add the last group if it exists
    if current_group:
        groups.append(current_group)

    return groups

def retrieve_project_details(text_ls):

    for i in text_ls:
        if 'Mature' in i or 'Non-mature' in i or 'mature' in i or 'non-mature' in i:
            name = i.split('(')[0].strip()
            details = i.split('(')[1].split(')')[0].strip()
            details = details.split(',')
            return name, details


In [207]:
df = pd.DataFrame()

df['Project'] = '' #to input project name
df['Town'] = '' #to input town
df['Category'] = '' #to input mature/non-mature
df['Number of Rooms'] = 0 #to input number of rooms
df['Num of Units'] = 0 #to input number of units
df['Number of Applications'] = 0 #to input number of applications

df

,Project,Town,Category,Number of Rooms,Num of Units,Number of Applications


In [208]:
for x in range(4,20):

    xpath = f'//*[@id="__next"]/main/div[1]/div[4]/div[1]/article/div[1]/figure[{x}]/div'

    element = driver.find_element(By.XPATH, xpath)
    text_body = element.text
    result = separate_by_mature(text_body.split('\n'))
    print(x, result)

    for i in result:
        
        project_details,project_dict = extract_relevant (i)
        if project_details == 0:
            continue
        for j in project_dict:
            if j == 0:
                continue
            df = pd.concat([df, pd.DataFrame(
                {'Project': [project_details[0]],
                'Town': [project_details[1][0]] if len(project_details[1]) > 1 else '',
                'Category': [project_details[1][1]] if len(project_details[1]) > 1 else project_details[1][0],
                'Number of Rooms': [j],
                'Num of Units': (project_dict[j][0]),
                'Number of Applications': [project_dict[j][1]]
                })],
                ignore_index=True)

4 [['Canberra Vista (Sembawang, Non-mature)', 'No. of units (3-, 4- and 5-Room)3-room: 124', '', '4-room: 385', '', '5-room/ 3Gen: 266', '', '', '', 'Total rooms:775', 'No. of applicants', '3-room: 847', '4-room: 3,048', '5-room/ 3Gen: 3,738', '= 7,633', 'Avg. application rate', '3-room: 6.8', '4-room: 7.9', '5-room/ 3Gen: 14.1', '= 9.8'], ['Toa Payoh Ridge / Kim Keat Ripples (Toa Payoh, Mature)', 'No. of units (3-, 4- and 5-Room)', '3-room: 102', '4-room: 1,211', '= 1,313', 'No. of applicants', '3-room: 935', '4-room: 11,684', '= 12,619', 'Avg. application rate', '3-room: 9.2', '4-room: 9.6', '= 9.6']]
5 [['Keat Hong Verge (Choa Chu Kang, Non-mature)', 'No. of units (3-, 4- and 5-Room)', '3-room: 118', '4-room: 290', '= 408', 'No. of applicants', '3-room: 263', '4-rom: 683', '= 946', 'Avg. application rate', '3-room: 2.2', '4-room: 2.4', '= 2.3'], ['Parc Residences @ Tengah (Non-mature)', 'No. of units (3-, 4- and 5-Room)', '3-room: 99', '4-room: 281', '5-room: 184', '= 564', 'No. of 

In [205]:
extract_relevant(result[0])

(None, {3: [120, 535], 4: [766, 2823], 5: [610, 3189]})

In [165]:
extract_data(result[0][3])

(4, 290)

In [196]:
result[0]

['Lakeside View (Jurong West, Non-mature',
 'No. of units (3-, 4- and 5-Room)',
 '3-room: 101',
 '4 room: 246',
 '5-room: 214',
 '= 561',
 'No. of applicants',
 '3-room: 446',
 '4-room: 1,827',
 '5 room: 3,294',
 '= 5,567',
 'Avg. application rate',
 '3-room: 4.4',
 '4-room: 7.4',
 '5-room: 15.4',
 '= 9.9']

In [210]:
df.to_csv('BTO_2020_2024.csv', index=False)

In [ ]:
driver.quit()